In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import recbole
aisles = pd.read_csv("../aisles.csv")
departments = pd.read_csv("../departments.csv")

order_products_train = pd.concat([pd.read_csv("../order_products__train.csv"),pd.read_csv('../order_products__prior.csv')],axis=0)
orders = pd.read_csv("../orders.csv")
products = pd.read_csv("../products.csv")

In [42]:
products[["product_id","aisle_id","department_id"]].to_csv("products.item",index=False,sep="\t")

In [6]:
combined = pd.merge(order_products_train,orders,how="inner",left_on="order_id",right_on="order_id")
combined = pd.merge(products,combined,how="inner",left_on="product_id",right_on="product_id")
combined = pd.merge(aisles,combined,how="inner",left_on="aisle_id",right_on="aisle_id")
combined = pd.merge(departments,combined,how="inner",left_on="department_id",right_on="department_id")
combined.columns

Index(['department_id', 'department', 'aisle_id', 'aisle', 'product_id',
       'product_name', 'order_id', 'add_to_cart_order', 'reordered', 'user_id',
       'eval_set', 'order_number', 'order_dow', 'order_hour_of_day',
       'days_since_prior_order'],
      dtype='object')

In [12]:
combined["days_since_prior_order"].fillna(30,inplace=True)

In [29]:
counts = combined["user_id"].value_counts()
counts[counts<4].shape

(99,)

In [39]:
uniques = combined.groupby("user_id")['product_id'].nunique()
combined_subset = combined[combined["user_id"].isin(uniques[uniques>1].index)]
combined_subset.shape

(33817284, 15)

In [35]:
combined_subset[["user_id","product_id","reordered","order_number","order_dow","eval_set"]].head()

,user_id,product_id,reordered,order_number,order_dow,eval_set
0,119781,118,1,19,4,train
1,58783,118,0,17,0,train
2,16301,118,1,21,4,train
3,22716,118,1,10,0,train
4,123212,118,0,22,4,train


In [41]:
combined_subset[combined_subset["user_id"]==48975].sort_values(by="order_number")

,department_id,department,aisle_id,aisle,product_id,product_name,order_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order


In [45]:
combined_subset[["user_id","product_id","reordered","order_number","order_dow","eval_set"]].to_csv("orders.inter",index=False,sep="\t")

In [ ]:
from recbole.config import Config
from recbole.data import create_dataset, data_preparation

if __name__ == '__main__':
    config = Config(model='BPR', dataset='ml-1m', config_file_list=['ml-1m.yaml'])
    dataset = create_dataset(config)